# Dengue Dataset: Correlation Analysis with Outcome Target

This notebook analyzes correlations between features and the Outcome variable in the Dengue dataset. We'll identify which features are most strongly related to disease outcome.

**Dataset columns:**
- Gender, Age, NS1, IgG, IgM (features)
- Area, AreaType, HouseType, District (categorical features)
- Outcome (target variable: 0=No, 1=Yes)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set_style("whitegrid")

## 1. Load and Explore the Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('../Dangue_Dataset/dataset.csv')

print("📊 Dataset Shape:", df.shape)
print("\n🔍 First 5 Rows:")
print(df.head())

print("\n📋 Data Types:")
print(df.dtypes)

print("\n❓ Missing Values:")
print(df.isnull().sum())

print("\n📈 Descriptive Statistics:")
print(df.describe())

## 2. Data Preprocessing and Encoding

Encode categorical variables for correlation analysis:

In [ ]:
# Create a copy for encoding
df_encoded = df.copy()

# Encode categorical columns
le_dict = {}
categorical_cols = ['Gender', 'Area', 'AreaType', 'HouseType', 'District']

print("🔄 Encoding categorical variables:\n")
for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    le_dict[col] = le
    print(f"✓ {col}: {dict(enumerate(le.classes_))}")

print("\n✅ All columns are now numeric!")
print("\nEncoded Dataset (first 5 rows):")
print(df_encoded.head())
print("\nData types after encoding:")
print(df_encoded.dtypes)

## 3. Calculate Correlation Matrix

In [ ]:
# Calculate correlation matrix
correlation_matrix = df_encoded.corr()

print("📊 Full Correlation Matrix:")
print(correlation_matrix)

print("\n" + "="*60)
print("💡 Correlation Statistics:")
print("="*60)
print(f"Matrix Shape: {correlation_matrix.shape}")
print(f"All columns are numeric: {correlation_matrix.isnull().sum().sum() == 0}")

## 4. Visualize Correlations with Heatmap

In [ ]:
# Create correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, cbar_kws={'label': 'Correlation'}, 
            vmin=-1, vmax=1, linewidths=0.5, square=True)
plt.title('Correlation Matrix Heatmap\n(All Features vs Outcome)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("✅ Heatmap displayed!")

## 5. Analyze Correlations with Target Variable (Outcome)

In [ ]:
# Extract correlations with Outcome (excluding Outcome itself)
outcome_correlations = correlation_matrix['Outcome'].drop('Outcome').sort_values(ascending=False)

print("🎯 CORRELATIONS WITH OUTCOME (TARGET):")
print("="*60)
print(outcome_correlations)

print("\n📊 Sorted Correlations (Strongest to Weakest):")
print("="*60)
for feature, corr in outcome_correlations.items():
    bar_length = int(abs(corr) * 30)
    bar = '█' * bar_length
    direction = "↑ positive" if corr > 0 else "↓ negative"
    print(f"{feature:15} {corr:7.4f}  {bar}  {direction}")

# Separate positive and negative correlations
positive_corr = outcome_correlations[outcome_correlations > 0].sort_values(ascending=False)
negative_corr = outcome_correlations[outcome_correlations < 0].sort_values(ascending=True)

print("\n" + "="*60)
print("✅ STRONGEST POSITIVE CORRELATIONS (increase with outcome):")
print("="*60)
if len(positive_corr) > 0:
    for feature, corr in positive_corr.items():
        print(f"  {feature:15}: {corr:7.4f}")
else:
    print("  None")

print("\n" + "="*60)
print("❌ STRONGEST NEGATIVE CORRELATIONS (decrease with outcome):")
print("="*60)
if len(negative_corr) > 0:
    for feature, corr in negative_corr.items():
        print(f"  {feature:15}: {corr:7.4f}")
else:
    print("  None")

In [ ]:
# Visualize correlations with Outcome as bar chart
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in outcome_correlations.values]
ax.barh(range(len(outcome_correlations)), outcome_correlations.values, color=colors, alpha=0.7)
ax.set_yticks(range(len(outcome_correlations)))
ax.set_yticklabels(outcome_correlations.index)
ax.set_xlabel('Correlation Coefficient', fontsize=12, fontweight='bold')
ax.set_title('Feature Correlations with Outcome\n(Green=Positive, Red=Negative)', 
             fontsize=13, fontweight='bold')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(outcome_correlations.values):
    ax.text(v + 0.01 if v > 0 else v - 0.01, i, f'{v:.4f}', 
            va='center', ha='left' if v > 0 else 'right', fontweight='bold')

plt.tight_layout()
plt.show()

print("✅ Bar chart displayed!")

## 6. Statistical Summary of Key Correlations

In [ ]:
# Calculate p-values for correlations
from scipy.stats import pearsonr

correlation_stats = []
for feature in df_encoded.columns:
    if feature != 'Outcome':
        corr_coef, p_value = pearsonr(df_encoded[feature], df_encoded['Outcome'])
        correlation_stats.append({
            'Feature': feature,
            'Correlation': corr_coef,
            'P-Value': p_value,
            'Abs_Correlation': abs(corr_coef),
            'Significant': 'Yes' if p_value < 0.05 else 'No'
        })

# Create DataFrame
stats_df = pd.DataFrame(correlation_stats).sort_values('Abs_Correlation', ascending=False)

print("📊 CORRELATION STATISTICS WITH P-VALUES:")
print("="*90)
print(stats_df.to_string(index=False))

print("\n" + "="*90)
print("🎯 KEY FINDINGS:")
print("="*90)

# Strongest correlations
print("\n🔴 STRONGEST PREDICTIVE FEATURES (by absolute correlation):")
top_features = stats_df.head(5)
for idx, row in top_features.iterrows():
    strength = "Very Strong" if abs(row['Correlation']) > 0.7 else \
               "Strong" if abs(row['Correlation']) > 0.5 else \
               "Moderate" if abs(row['Correlation']) > 0.3 else "Weak"
    sig = "✓" if row['Significant'] == 'Yes' else "✗"
    print(f"  {sig} {row['Feature']:15}: r={row['Correlation']:7.4f} (p={row['P-Value']:.2e}) - {strength}")

# Significant features
print("\n✅ STATISTICALLY SIGNIFICANT FEATURES (p < 0.05):")
sig_features = stats_df[stats_df['Significant'] == 'Yes']
if len(sig_features) > 0:
    for idx, row in sig_features.iterrows():
        print(f"  • {row['Feature']:15}: r={row['Correlation']:7.4f} (p={row['P-Value']:.2e})")
else:
    print("  None")

In [ ]:
# Create summary visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Top 5 features by absolute correlation
ax1 = axes[0, 0]
top5 = stats_df.head(5)
colors_top5 = ['green' if x > 0 else 'red' for x in top5['Correlation']]
ax1.barh(range(len(top5)), top5['Correlation'].values, color=colors_top5, alpha=0.7)
ax1.set_yticks(range(len(top5)))
ax1.set_yticklabels(top5['Feature'].values)
ax1.set_xlabel('Correlation')
ax1.set_title('Top 5 Features by Correlation', fontweight='bold')
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.8)

# 2. Distribution of correlations
ax2 = axes[0, 1]
ax2.hist(stats_df['Correlation'].values, bins=15, edgecolor='black', alpha=0.7, color='skyblue')
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero correlation')
ax2.set_xlabel('Correlation Coefficient')
ax2.set_ylabel('Number of Features')
ax2.set_title('Distribution of Correlations', fontweight='bold')
ax2.legend()

# 3. P-value significance
ax3 = axes[1, 0]
sig_count = len(stats_df[stats_df['Significant'] == 'Yes'])
non_sig_count = len(stats_df[stats_df['Significant'] == 'No'])
labels = [f'Significant\n(p<0.05)\n{sig_count}', f'Not Significant\n(p≥0.05)\n{non_sig_count}']
colors_pie = ['green', 'red']
ax3.pie([sig_count, non_sig_count], labels=labels, colors=colors_pie, autopct='%1.1f%%', 
        startangle=90, textprops={'fontsize': 10, 'weight': 'bold'})
ax3.set_title('Significant vs Non-significant Features', fontweight='bold')

# 4. Summary statistics
ax4 = axes[1, 1]
ax4.axis('off')
summary_text = f"""
CORRELATION ANALYSIS SUMMARY

Total Features Analyzed: {len(stats_df)}
Significant Features: {sig_count} ({sig_count/len(stats_df)*100:.1f}%)
Non-significant Features: {non_sig_count} ({non_sig_count/len(stats_df)*100:.1f}%)

Mean Absolute Correlation: {stats_df['Abs_Correlation'].mean():.4f}
Max Absolute Correlation: {stats_df['Abs_Correlation'].max():.4f}
Min Absolute Correlation: {stats_df['Abs_Correlation'].min():.4f}

Positive Correlations: {len(stats_df[stats_df['Correlation'] > 0])}
Negative Correlations: {len(stats_df[stats_df['Correlation'] < 0])}
"""
ax4.text(0.1, 0.5, summary_text, fontsize=11, family='monospace', 
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("✅ Summary visualization displayed!")

## 7. Key Insights and Recommendations

### Summary of Findings:

**Biological/Medical Features (Most Important):**
- **NS1**: Non-structural protein 1 - key marker for dengue diagnosis
- **IgG**: Immunoglobulin G antibodies - late-stage immune response
- **IgM**: Immunoglobulin M antibodies - early-stage immune response

These three parameters typically show strong correlations with dengue infection status.

**Demographic Factors:**
- **Age**: May show patterns in susceptibility
- **Gender**: Could indicate differential risk patterns

**Environmental Factors:**
- **Area, AreaType, HouseType, District**: Geographic and housing conditions affecting transmission

### Interpretation:
- High positive correlation → Feature increases with disease presence
- High negative correlation → Feature decreases with disease presence
- Low/near-zero correlation → Feature has minimal predictive value

### Recommendations for Model Building:
1. **Use high-correlation features first** for initial models
2. **Remove highly correlated features** to avoid multicollinearity (check feature-to-feature correlations)
3. **Test non-linear relationships** for features with weak linear correlations
4. **Consider interaction effects** between medical and environmental features
5. **Validate on test set** to ensure correlations translate to good predictions